In [4]:
pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.1/875.1 kB 45.1 MB/s eta 0:00:00


In [5]:
import gdown
import os
import yaml
import shutil
import torch
from ultralytics import YOLO
from sklearn.model_selection import train_test_split
# методы для отрисовки изображений
from PIL import Image

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
# Функция для установки seed
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

# Установите значение seed
set_seed(42)

In [ ]:
#BB РАЗМЕТКА https://drive.google.com/file/d/1ncEDgS7YzUdPYjVab7fl4lvdjL9uHEA_/view?usp=sharing
!gdown 1ncEDgS7YzUdPYjVab7fl4lvdjL9uHEA_

Downloading...
From (original): https://drive.google.com/uc?id=1ncEDgS7YzUdPYjVab7fl4lvdjL9uHEA_
From (redirected): https://drive.google.com/uc?id=1ncEDgS7YzUdPYjVab7fl4lvdjL9uHEA_&confirm=t&uuid=1a39dd7c-3563-4f30-b0e0-ea5f912ae0f0
To: /content/dataset16.zip
100% 302M/302M [00:03<00:00, 92.1MB/s]


In [ ]:
# АРХИВ С ПОДГОТОВЛЕНЫМИ ИЗОБРАЖЕНИЯМИ
!unzip -q  "/content/dataset16.zip" -d /content/Dataset


# Папка с папками картинок, рассортированных по категориям
IMAGE_PATH = '/content/Dataset'

# Получение списка папок, находящемуся по адресу в скобках
os.listdir(IMAGE_PATH)

['labels', 'images']

In [ ]:

os.makedirs('/content/Data_yola', exist_ok=True)

In [ ]:
# Путь к папкам с изображениями и аннотациями
images_dir = f'{IMAGE_PATH}/images'
labels_dir = f'{IMAGE_PATH}/labels'

# Путь к папкам для train и val
train_images_dir = '/content/Data_yola/images/train'
train_labels_dir = '/content/Data_yola/labels/train'
val_images_dir = '/content/Data_yola/images/val'
val_labels_dir = '/content/Data_yola/labels/val'

# Создание папок для train и val
os.makedirs(train_images_dir, exist_ok=True)
os.makedirs(train_labels_dir, exist_ok=True)
os.makedirs(val_images_dir, exist_ok=True)
os.makedirs(val_labels_dir, exist_ok=True)

# Словари для хранения списка файлов по классам
class_files = {}

# Классифицируйте файлы по классам
for filename in os.listdir(images_dir):
    if filename.endswith('.jpg'):
        class_name = filename.split('_')[0]
        label_file = filename.replace('.jpg', '.txt')

        if class_name not in class_files:
            class_files[class_name] = {'images': [], 'labels': []}

        class_files[class_name]['images'].append(filename)
        class_files[class_name]['labels'].append(label_file)

# Функция для разделения данных на train и val
def split_data(class_files, train_size=0.9):
    train_files = {'images': [], 'labels': []}
    val_files = {'images': [], 'labels': []}

    for class_name, files in class_files.items():
        train_images, val_images = train_test_split(files['images'], train_size=train_size, random_state=42)
        train_labels = [filename.replace('.jpg', '.txt') for filename in train_images]
        val_labels = [filename.replace('.jpg', '.txt') for filename in val_images]

        train_files['images'].extend(train_images)
        train_files['labels'].extend(train_labels)
        val_files['images'].extend(val_images)
        val_files['labels'].extend(val_labels)

    return train_files, val_files

# Разделите данные и скопируйте файлы
train_files, val_files = split_data(class_files)

def copy_files(files, src_dir, dest_dir):
    for file in files:
        src = os.path.join(src_dir, file)
        dst = os.path.join(dest_dir, file)
        shutil.copy(src, dst)

# Копируем файлы в папки train
copy_files(train_files['images'], images_dir, train_images_dir)
copy_files(train_files['labels'], labels_dir, train_labels_dir)

# Копируем файлы в папки val
copy_files(val_files['images'], images_dir, val_images_dir)
copy_files(val_files['labels'], labels_dir, val_labels_dir)

print(f'Разделение завершено. Обучающие файлы: {len(train_files["images"])}, Валидационные файлы: {len(val_files["images"])}')

Разделение завершено. Обучающие файлы: 2193, Валидационные файлы: 245


In [ ]:
#СОЗДАЕМ YAML

def create_yaml(data_dir, yaml_path):


    # Получаем список всех файлов в папках
    def get_file_names(directory, extension='.jpg'):
        return [f for f in os.listdir(directory) if f.endswith(extension)]

    train_images = get_file_names(train_images_dir)
    val_images = get_file_names(val_images_dir)

    # Извлекаем уникальные имена классов из аннотаций
    def get_classes_from_labels(labels_dir):
        classes = set()
        for file_name in os.listdir(labels_dir):
            if file_name.endswith('.txt'):
                with open(os.path.join(labels_dir, file_name), 'r') as f:
                    for line in f:
                        class_id = int(line.split()[0])
                        classes.add(class_id)
        return classes

    class_ids = get_classes_from_labels(train_labels_dir)
    class_names = [f'class_{i}' for i in sorted(class_ids)]

    # Создание YAML-файла
    data = {
        'train': train_images_dir,
        'val': val_images_dir,
        'nc': len(class_names),
        'names': class_names
    }

    with open(yaml_path, 'w') as yaml_file:
        yaml.dump(data, yaml_file, default_flow_style=False)

    print(f'YAML файл создан по пути: {yaml_path}')

# Укажите путь к директории данных и путь для YAML файла
data_directory = '/content/Dataset'
yaml_file_path = '/content/Data_yolo.yaml'

create_yaml(data_directory, yaml_file_path)

YAML файл создан по пути: /content/Data_yolo.yaml


In [ ]:
# Create a new YOLOv8n-BB model from scratch
model = YOLO("yolov8n.yaml")


In [ ]:
pip install --upgrade ultralytics

In [ ]:
pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.8/362.8 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.4 MB/s eta 0:00:00


In [ ]:
import optuna
from ultralytics import YOLO


def objective(trial):
    # Определение гиперпараметров для текущего пробного эксперимента
    lr0 = trial.suggest_loguniform('lr0', 1e-6, 1e-1)
    batch = trial.suggest_categorical('batch', [16, 32])
    momentum = trial.suggest_uniform('momentum', 0.5, 0.95)
    weight_decay = trial.suggest_loguniform('weight_decay', 1e-6, 1e-2)
    dropout = trial.suggest_uniform('dropout', 0.1, 0.5)
    optimizer = trial.suggest_categorical('optimizer', ['Adam', 'AdamW'])
    hsv_h = trial.suggest_float('hsv_h', 0.001, 1.0, log=True)
    hsv_s = trial.suggest_float('hsv_s', 0.1, 1.0, log=True)
    hsv_v = trial.suggest_float('hsv_v', 0.1, 1.0, log=True)


    # Создание и тренировка модели
    model = YOLO('yolov8n.pt')

    # Тренировка модели
    results = model.train(
        data="/content/Data_yolo.yaml",
        epochs=40,
        imgsz=640,
        batch=batch,
        lr0=lr0,
        momentum=momentum,
        weight_decay=weight_decay,
        dropout=dropout,
        optimizer=optimizer,
        hsv_h=hsv_h,
        hsv_s=hsv_s,
        hsv_v=hsv_v,
    )

    # Получаем метрики модели
    metrics = model.metrics

    # Извлечение нужных метрик
    metrics_dict = metrics.results_dict
    mAP50 = metrics_dict['metrics/mAP50(B)']  # Средняя точность при IoU=0.5

    return mAP50

# Создание исследования для оптимизации
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=1)  # Количество проб

# Вывод наилучших гиперпараметров
print("Best hyperparameters: ", study.best_params)

[I 2024-09-08 06:40:46,705] A new study created in memory with name: no-name-95998930-3667-47b6-b6be-b72f291e9fb8


Ultralytics YOLOv8.2.90 🚀 Python-3.10.12 torch-2.4.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/Data_yolo.yaml, epochs=40, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train9, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.33729326751034244, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_c

train: Scanning /content/Data_yola/labels/train.cache... 2193 images, 3 backgrounds, 1 corrupt: 100%|██████████| 2193/2193 [00:00<?, ?it/s]

train: WARNING ⚠️ /content/Data_yola/images/train/2Class_00187.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0286]
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /content/Data_yola/labels/val.cache... 245 images, 0 backgrounds, 1 corrupt: 100%|██████████| 245/245 [00:00<?, ?it/s]

...


40 epochs completed in 0.561 hours.
Optimizer stripped from runs/detect/train9/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train9/weights/best.pt, 6.2MB

Validating runs/detect/train9/weights/best.pt...
Ultralytics YOLOv8.2.90 🚀 Python-3.10.12 torch-2.4.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3,006,428 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:06<00:00,  1.18it/s]


                   all        244        486      0.851      0.652      0.775      0.554
               class_0         51         73      0.925      0.674      0.808      0.591
               class_1         47        160      0.902        0.6      0.742      0.529
               class_2        106        154       0.76       0.74      0.844      0.632
               class_3         43         99      0.819      0.593      0.707      0.462
Speed: 0.6ms preprocess, 4.0ms inference, 0.0ms loss, 6.6ms postprocess per image
Results saved to runs/detect/train9


[I 2024-09-08 07:14:47,327] Trial 0 finished with value: 0.775404064658332 and parameters: {'lr0': 0.0013177052265558483, 'batch': 16, 'momentum': 0.8893119045334239, 'weight_decay': 4.3756645434811967e-05, 'dropout': 0.33729326751034244, 'optimizer': 'AdamW', 'hsv_h': 0.18066223761178896, 'hsv_s': 0.21508468576649098, 'hsv_v': 0.3997828610133023}. Best is trial 0 with value: 0.775404064658332.


Best hyperparameters:  {'lr0': 0.0013177052265558483, 'batch': 16, 'momentum': 0.8893119045334239, 'weight_decay': 4.3756645434811967e-05, 'dropout': 0.33729326751034244, 'optimizer': 'AdamW', 'hsv_h': 0.18066223761178896, 'hsv_s': 0.21508468576649098, 'hsv_v': 0.3997828610133023}


In [ ]:
# Получение текущего seed для CUDA
import torch
cuda_seed = torch.cuda.initial_seed()

print(f"Текущий seed для CUDA: {cuda_seed}")

Текущий seed для CUDA: 0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
model = YOLO('/content/runs/detect/train9/weights/best.pt')

# Выполните оценку на валидационном наборе данных
metrics = model.val()

# Вывод метрик
print(metrics)

Ultralytics YOLOv8.2.90 🚀 Python-3.10.12 torch-2.4.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3,006,428 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /content/Data_yola/labels/val.cache... 245 images, 0 backgrounds, 1 corrupt: 100%|██████████| 245/245 [00:00<?, ?it/s]

val: WARNING ⚠️ /content/Data_yola/images/val/3Class_00012.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0296]



/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 16/16 [00:05<00:00,  2.79it/s]


                   all        244        486       0.85       0.65      0.775      0.553
               class_0         51         73      0.925      0.673      0.808      0.591
               class_1         47        160      0.902        0.6      0.742      0.529
               class_2        106        154      0.763       0.74      0.844      0.632
               class_3         43         99      0.808      0.586      0.708      0.461
Speed: 0.5ms preprocess, 5.6ms inference, 0.0ms loss, 4.1ms postprocess per image
Results saved to runs/detect/val
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f8436437c40>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0

In [ ]:
import matplotlib.pyplot as plt


def display_images_from_folder_sorted(folder_path):
    """Отображает все изображения из указанной папки с подписями и сортировкой по названию с помощью matplotlib."""
    # Получаем список всех файлов в папке и сортируем их по названию
    file_names = sorted([f for f in os.listdir(folder_path) if f.endswith(('.png', '.jpg', '.jpeg'))])

    if not file_names:
        print("В папке нет изображений.")
        return

    num_images = len(file_names)
    num_cols = 1  # Количество столбцов (уменьшение увеличит размер изображения)
    num_rows = (num_images + num_cols - 1) // num_cols  # Расчет количества строк

    # Создаем фигуру и оси, увеличиваем размер фигуры
    fig, axes = plt.subplots(num_rows, num_cols, figsize=(15, num_rows * 7))

    # Выравниваем оси
    axes = axes.flatten()

    for i, file_name in enumerate(file_names):
        image_path = os.path.join(folder_path, file_name)
        # Открываем изображение
        image = Image.open(image_path)
        # Отображаем изображение на соответствующей оси
        axes[i].imshow(image)
        axes[i].set_title(file_name, fontsize=12)
        axes[i].axis('off')  # Скрываем оси

    # Убираем пустые оси, если они есть
    for ax in axes[num_images:]:
        ax.axis('off')

    plt.tight_layout()
    plt.show()

# Укажите путь к папке с изображениями
folder_path = '/content/runs/detect/train9'

# Выполняем отображение изображений
display_images_from_folder_sorted(folder_path)

Output hidden; open in https://colab.research.google.com to view.

In [1]:
from google.colab import drive
#drive.mount('/content/drive')

In [2]:
#Скачиваем видео и  веса
!gdown 1ErRtAyOsaY18gIOlsQgl7WO0GPQ6_3nw
!gdown 1zDbfknsUz9fgnQkEvSWd5eFU_tWuyUfx

Downloading...
From: https://drive.google.com/uc?id=1ErRtAyOsaY18gIOlsQgl7WO0GPQ6_3nw
To: /content/VOP.mp4
100% 96.7M/96.7M [00:04<00:00, 23.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1zDbfknsUz9fgnQkEvSWd5eFU_tWuyUfx
To: /content/yolo8n_best27.pt
100% 6.25M/6.25M [00:00<00:00, 16.9MB/s]


In [6]:
import cv2
from ultralytics import YOLO
import numpy as np
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

# Загружаем модель YOLO
model = YOLO('yolo8n_best27.pt')

# Открываем видеофайл
video_path = 'VOP.mp4'
cap = cv2.VideoCapture(video_path)

# Получаем параметры исходного видео
fps = cap.get(cv2.CAP_PROP_FPS)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Размер блоков
block_size = 640

# Создаем VideoWriter для сохранения нового видео
out = cv2.VideoWriter('output_with_predictions1.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

frame_count = 0
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Функция для параллельной обработки блока
def process_block(block):
    results = model.predict(block, verbose=False)
    return results[0].plot()

with tqdm(total=total_frames) as pbar:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1

        # Обрабатываем каждый 10-й кадр
        if frame_count % 10 == 0:
            height, width, _ = frame.shape
            num_blocks_x = width // block_size
            num_blocks_y = height // block_size
            remainder_x = width % block_size
            remainder_y = height % block_size

            blocks = []

            # Разбиваем кадр на блоки
            for i in range(num_blocks_y):
                for j in range(num_blocks_x):
                    x = j * block_size
                    y = i * block_size
                    block = frame[y:y + block_size, x:x + block_size]
                    blocks.append((block, (y, x)))

            if remainder_x > 0:
                for i in range(num_blocks_y):
                    x = num_blocks_x * block_size
                    y = i * block_size
                    block = frame[y:y + block_size, x:width]
                    blocks.append((block, (y, x)))

            if remainder_y > 0:
                for j in range(num_blocks_x):
                    x = j * block_size
                    y = num_blocks_y * block_size
                    block = frame[y:height, x:x + block_size]
                    blocks.append((block, (y, x)))

            if remainder_x > 0 and remainder_y > 0:
                x = num_blocks_x * block_size
                y = num_blocks_y * block_size
                block = frame[y:height, x:width]
                blocks.append((block, (y, x)))

            # Параллельная обработка блоков
            with ThreadPoolExecutor() as executor:
                futures = {executor.submit(process_block, block): idx for idx, (block, coords) in enumerate(blocks)}
                processed_blocks = [None] * len(blocks)

                for future in as_completed(futures):
                    index = futures[future]  # Получаем индекс блока
                    processed_block = future.result()
                    processed_blocks[index] = processed_block

            # Создаем пустой кадр для выходного видео
            output_frame = np.zeros_like(frame)

            # Вставляем обработанные блоки на соответствующие места
            for (block, (y, x)), processed_block in zip(blocks, processed_blocks):
                if processed_block is not None:
                    h, w, _ = processed_block.shape
                    output_frame[y:y + h, x:x + w] = processed_block

            # Записываем обработанный кадр в видео
            out.write(output_frame)
        else:
            # Если кадр не обрабатывается, просто записываем его как есть
            out.write(frame)

        pbar.update(1)

# Освобождаем ресурсы
cap.release()
out.release()




  0%|          | 5/1661 [00:00<01:12, 22.84it/s]

Ultralytics YOLOv8.2.103 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Ultralytics YOLOv8.2.103 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Ultralytics YOLOv8.2.103 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3,006,428 parameters, 0 gradients, 8.1 GFLOPs
Model summary (fused): 168 layers, 3,006,428 parameters, 0 gradients, 8.1 GFLOPs
Model summary (fused): 168 layers, 3,006,428 parameters, 0 gradients, 8.1 GFLOPs


100%|██████████| 1661/1661 [01:01<00:00, 26.81it/s]


[Файл output все кадры](https://colab.research.google.com/drive/11QUwtSdAhUqWqytgtguOfZ_oDH6ixVLY#scrollTo=gS8XntRsFOlH&line=1&uniqifier=1)